# Calculate the percentage of anomaly
#### Jose Valles (jose.valles.leon@gmail.com)

### Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
import numpy as np
import calendar

sns.set()

from IPython.display import HTML

In [14]:
import geopandas as gpd

Import basin level code 2 and 3

In [2]:
BASIN_LEVEL3 = pd.read_csv(f'../output_modelo/cuenca_nivel3.csv',index_col="Codigo")
BASIN_LEVEL2 = pd.read_csv(f'../output_modelo/cuenca_nivel2.csv',index_col="Codigo")

In [3]:
def importmodelvariable(model_variable):
    df = pd.read_csv(f'../output_modelo/{model_variable}.csv')
    df = df.rename(columns={'-1': 'year','-1.1':'month'})
    df['date'] = pd.to_datetime(dict(year=df['year'],month=df['month'],day=1))
    df = df.set_index('date')
    df['days_in_month'] = df.index.days_in_month
    return df

def convertRunoff2Discharge(df_runoff):
    df_runoff_selected = df_runoff.drop(['year','month','days_in_month'],axis=1)
    df_discharge = pd.DataFrame(df_runoff_selected.values*1000*BASIN_LEVEL3.values,columns=df_runoff_selected.columns)
    df_discharge['days_in_month'] = df_runoff['days_in_month'].values
    df_discharge = df_discharge.loc[:, df_discharge.columns != 'days_in_month'].divide(df_discharge["days_in_month"]*24*3600, axis="index")
    df_discharge['date'] = df_runoff.index.values
    df_discharge = df_discharge.set_index('date')
    df_discharge['year'] = df_runoff['year'].values
    df_discharge['month'] = df_runoff['month'].values
    return df_discharge

def defineHydroSOScategory(VARIABLE_MENSUAL,VARIABLE_AVERAGE,VARIABLE):
    # create empty columns in the dataframe
    VARIABLE_MENSUAL['mean'] = np.nan
    VARIABLE_MENSUAL['average_percentage'] = np.nan
    VARIABLE_MENSUAL['rank_average'] = np.nan
    VARIABLE_MENSUAL['non_missing'] = np.nan


    for i in range(len(VARIABLE_MENSUAL)):
        # Extract the current month 
        m = VARIABLE_MENSUAL.month[i]
        # Extract the current year
        y = VARIABLE_MENSUAL.year[i]
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'rank_average']  = VARIABLE_MENSUAL.query('month==@m')[VARIABLE].rank()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'non_missing']  = VARIABLE_MENSUAL.query('month==@m')[VARIABLE].notnull().sum()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'mean'] = VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'average_percentage'] = (VARIABLE_MENSUAL[VARIABLE][i] - VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()) / VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()

    VARIABLE_MENSUAL['percentile'] = VARIABLE_MENSUAL['rank_average']/(VARIABLE_MENSUAL['non_missing']+1)
    criteria = [VARIABLE_MENSUAL['percentile'].between(0.87,1.00),
            VARIABLE_MENSUAL['percentile'].between(0.72,0.87),
            VARIABLE_MENSUAL['percentile'].between(0.28,0.72),
            VARIABLE_MENSUAL['percentile'].between(0.13,0.28),
            VARIABLE_MENSUAL['percentile'].between(0.00,0.13)]

    values = ['High flow','Above normal','Normal range','Below normal','Low flow']

    VARIABLE_MENSUAL['percentile_range'] = np.select(criteria,values,None)
    return VARIABLE_MENSUAL

In [4]:
hydrological_variable = ['Escorrentia_total','Escorrentia_sup','Escorrentia_sub','Pmedias','ETR','HumedadSuelo']

for hydro in hydrological_variable:
    if hydro == "Escorrentia_total":
        RUNOFF_total = importmodelvariable(hydro)
    elif hydro == "Pmedias":
        PRECIP = importmodelvariable(hydro)
    elif hydro == "ETR":
        ETR = importmodelvariable(hydro)
    elif hydro == "HumedadSuelo":
        SM = importmodelvariable(hydro)
    elif hydro == 'Escorrentia_sup':
        RUNOFF_sup = importmodelvariable(hydro)
    elif hydro == 'Escorrentia_sub':
        RUNOFF_sub = importmodelvariable(hydro)

In [5]:
HTML(PRECIP.tail(6).to_html(index=False))

year,month,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,days_in_month
2023,1,52.394,34.940,17.9440,22.6920,23.225,23.3880,39.061,25.645,22.292,22.514,22.210,22.656,23.565,24.501,25.486,40.691,61.928,67.762,65.232,60.097,75.129,61.356,72.058,78.860,73.224,88.915,85.099,83.305,63.724,54.363,42.155,48.466,56.359,54.134,71.256,64.486,65.509,63.568,61.926,57.166,53.239,47.254,52.506,37.423,37.633,39.695,49.086,50.921,40.914,40.162,45.049,44.106,38.3700,36.3210,43.214,51.991,50.3580,37.9080,46.3940,47.3570,47.6550,48.2470,52.5780,40.1350,47.9330,48.9680,55.2540,54.4900,55.3460,55.2180,51.9330,54.4010,54.8040,54.5980,55.5890,55.5000,55.5070,55.5120,27.9580,29.8610,30.0670,29.9650,34.7550,39.7200,49.7460,48.2250,50.5960,55.2570,51.9850,46.7410,38.3860,24.2540,15.8850,17.5450,9.0953,6.9817,7.2451,4.7307,3.7247,3.4724,4.2160,5.8310,5.2360,5.9336,5.5642,5.5587,5.5952,5.5946,10.400,7.5986,6.9292,7.9384,10.982,5.8207,6.6777,9.8221,8.4946,6.2059,8.6135,10.2600,10.651,12.072,11.369,11.600,11.374,14.275,12.988,11.538,11.074,11.970,9.0925,8.0604,7.0856,11.635,13.607,32.391,34.481,31.769,48.020,54.874,22.809,27.123,25.726,25.323,24.013,23.177,23.067,21.518,21.081,20.704,18.891,21.774,29.433,27.841,22.760,20.193,22.343,15.446,19.182,12.618,10.692,9.0251,15.046,9.9731,13.571,11.236,8.4611,12.126,11.514,14.591,16.031,18.072,11.974,12.118,22.118,17.244,37.226,28.732,21.744,17.899,39.655,52.415,52.753,52.273,24.188,26.035,23.268,23.725,25.079,26.536,26.967,28.027,20.745,70.966,70.105,65.323,71.485,80.844,97.025,125.520,85.872,107.420,99.637,114.840,123.910,119.710,116.670,97.911,108.020,84.675,58.781,36.212,34.003,78.359,67.131,21.395,16.873,16.583,14.436,16.385,10.886,8.006,18.764,4.5224,19.230,21.375,21.140,19.509,19.830,20.913,28.607,9.650,8.0887,14.524,10.238,9.0989,8.7006,18.079,15.354,18.215,21.073,33.818,23.255,40.686,45.926,32.939,46.948,43.354,47.663,50.161,54.040,50.0860,43.1300,52.9820,14.360,13.439,10.869,7.6130,7.6961,8.2220,7.0541,7.0112,6.9143,6.6375,9.6249,8.4784,7.9359,6.6119,6.0891,6.9143,7.6941,7.0218,8.0760,7.0991,7.6687,8.3153,7.8095,17.3740,17.2060,16.3090,18.7900,16.4230,12.494,10.3510,7.9614,8.1746,7.9024,8.4805,7.8232,8.5226,7.5974,8.0233,6.040,7.6110,7.0860,5.7661,31
2023,2,62.534,54.359,37.2720,20.0890,19.391,31.4080,52.318,61.657,62.940,66.643,64.139,63.924,59.893,58.074,60.931,67.065,72.136,75.320,75.090,73.890,73.737,70.222,75.153,76.667,77.573,81.521,82.404,81.071,67.068,56.744,51.745,52.661,57.936,48.108,72.682,56.646,56.087,47.128,49.636,50.111,54.826,58.798,78.502,75.930,69.964,68.244,76.374,67.279,67.400,68.212,64.649,63.702,70.0290,71.7190,63.995,35.637,16.9550,15.0260,12.6320,10.3150,6.5104,6.5628,3.5057,11.0900,7.2598,6.6304,5.2236,4.9600,5.1552,4.9916,4.5155,5.1249,6.7584,8.2308,5.8497,5.9369,5.8330,5.6149,25.4320,24.3410,25.5820,26.0990,23.8020,17.7670,11.2780,9.7067,10.5110,5.9616,9.2377,14.1

### Convert from runoff to discharge

In [6]:
DISCHARGE = convertRunoff2Discharge(RUNOFF_total)

In [7]:
HTML(DISCHARGE.tail(6).to_html(index=True))

,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,year,month
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.453056,0.545085,0.306605,0.061179,0.118196,0.490494,1.319778,0.378910,0.018268,0.142463,0.025560,0.079836,0.080402,0.023417,0.123512,0.758099,0.371073,0.429049,0.742762,0.315674,0.152890,0.714520,0.419133,1.112000,0.704442,3.745383,0.798269,0.834020,0.728443,0.257053,0.008858,0.232726,1.069116,0.303503,1.679057,0.750343,0.951297,0.078998,0.137301,0.230917,0.700032,0.220528,1.020576,1.432463,0.090413,0.062360,0.896310,0.328678,0.067713,0.099860,0.302148,0.213928,0.011243,0.088988,0.467934,0.004805,1.718869e-06,0.000785,5.881655e-06,3.630708e-06,2.623555e-08,2.002917e-06,2.900539e-06,5.063829e-06,2.718312e-06,5.437534e-07,4.202509e-07,3.793841e-07,9.018869e-07,9.120871e-07,1.741264e-06,2.056877e-08,8.347984e-07,6.943380e-07,1.036617e-06,8.709080e-08,2.546360e-07,1.688900e-06,9.506392e-06,2.928498e-06,1.732576e-05,1.170312e-05,2.299989e-05,2.393405e-05,7.855873e-07,1.298057e-05,4.949597e-07,2.511727e-07,7.939553e-08,4.889796e-07,1.900848e-05,3.282971e-05,1.312843e-05,1.208261e-06,3.799997e-06,2.272115e-06,1.677778e-06,0.000668,0.001073,0.002360,0.002806,0.022710,0.000275,0.055737,0.004928,0.001754,0.000937,0.001640,0.027794,0.008636,0.017248,0.013590,0.045536,0.014510,0.002311,0.003165,0.006111,0.009567,0.002161,0.007059,0.005470,0.137352,0.004454,0.013553,0.002483,0.280412,0.144323,0.048978,0.010050,0.317805,0.320804,0.023852,0.832903,0.071066,0.974728,0.099912,0.497759,1.400027,0.090664,0.090687,1.099409,0.036386,0.154766,0.152430,0.175150,0.357579,0.478902,0.031192,0.160280,0.411654,0.094211,0.369482,0.007107,0.091807,0.060141,0.244276,0.100962,0.052982,0.070875,0.017150,0.046922,0.132762,0.278026,0.351221,0.578970,0.071470,0.160992,0.181113,0.168864,0.239145,0.360476,0.774901,0.318975,0.366516,0.054451,1.222690,0.155346,0.319296,0.091348,0.514257,0.642154,0.069599,0.005422,0.341653,0.199870,0.628980,0.369247,1.050637,0.562534,0.717217,0.711192,0.595577,0.939738,0.079755,0.071810,0.218253,0.353898,0.233198,0.320052,0.583380,2.504578,0.092812,0.373305,0.643558,0.658032,0.971566,0.733234,0.036777,0.310582,0.307014,1.857988,1.303701,0.130018,0.388460,1.364824,0.280035,0.294118,0.620483,0.462387,0.424515,0.253044,0.004745,0.256174,0.041131,0.083377,0.834123,0.215731,0.116211,0.054525,0.175444,0.133563,0.368782,0.171417,0.032018,0.236033,0.008734,0.000184,0.068392,0.000313,0.022222,0.022747,0.002505,0.001317,0.018454,0.004594,0.803219,0.262926,0.007225,0.002581,0.098680,0.000693,0.001321,0.002805,0.000426,0.042825,0.138107,0.063012,0.004284,0.019403

### Pleasee select the runoff type for the analysis (RUNOFF_total, RUNOFF_sub, RUNOFF_sup)

In [8]:
# Select the runoff type 
RUNOFF = RUNOFF_total

### Select reference period from 'year_start' to 'year_end'

Select variable of interest

In [9]:
year_start = 1981
year_end = 2010
# Caudal
SELECTED_REF_DISCHARGE= DISCHARGE[(DISCHARGE['year'] >= year_start) & (DISCHARGE['year'] <= year_end)]
# Escorrentia tota
SELECTED_REF_RUNOFF = RUNOFF[(RUNOFF['year'] >= year_start) & (RUNOFF['year'] <= year_end)]
# Precip
SELECTED_REF_PRECIP = PRECIP[(PRECIP['year'] >= year_start) & (PRECIP['year'] <= year_end)]

## Calculate the percentage of anomaly for each basin code level 2 and a specific month and year

In [10]:
#--
month_analyis = 6
year_analysis = 2023

AVERAGE_PERCENTAGE = pd.DataFrame()
AVERAGE_PERCENTAGE['codigo'] = BASIN_LEVEL2.columns
AVERAGE_PERCENTAGE['discharge'] = np.nan
AVERAGE_PERCENTAGE['runoff'] = np.nan
AVERAGE_PERCENTAGE['precip'] = np.nan



for basin in BASIN_LEVEL2.columns:
    # filter the basin code bas
    filter_col = [col for col in DISCHARGE if col.startswith(str(basin))]
    # Select basins level 3 that belongs to level 2 basin
    # Discharge
    DISCHARGE_FILTER = DISCHARGE[filter_col]
    DISCHARGE_SELECTED = pd.DataFrame()
    DISCHARGE_SELECTED['year'] = DISCHARGE['year']
    DISCHARGE_SELECTED['month'] = DISCHARGE['month']    
    DISCHARGE_SELECTED['discharge'] = DISCHARGE_FILTER.sum(axis=1)
    DISCHARGE_REF = DISCHARGE_SELECTED[(DISCHARGE_SELECTED['year'] >= year_start) & (DISCHARGE_SELECTED['year'] <= year_end)]
    # Runoff
    RUNOFF_FILTER = RUNOFF[filter_col]
    RUNOFF_SELECTED = pd.DataFrame()
    RUNOFF_SELECTED['year'] = RUNOFF['year']
    RUNOFF_SELECTED['month'] = RUNOFF['month']    
    RUNOFF_SELECTED['runoff'] = RUNOFF_FILTER.mean(axis=1)
    RUNOFF_REF = RUNOFF_SELECTED[(RUNOFF_SELECTED['year'] >= year_start) & (RUNOFF_SELECTED['year'] <= year_end)]
    # Precip
    PRECIP_FILTER = PRECIP[filter_col]
    PRECIP_SELECTED = pd.DataFrame()
    PRECIP_SELECTED['year'] = PRECIP['year']
    PRECIP_SELECTED['month'] = PRECIP['month']    
    PRECIP_SELECTED['precip'] = PRECIP_FILTER.mean(axis=1)
    PRECIP_REF = PRECIP_SELECTED[(PRECIP_SELECTED['year'] >= year_start) & (PRECIP_SELECTED['year'] <= year_end)]
    # CALCULATE CLIMATOLOGICAL NORMALS
    # Discharge
    DISCHARGE_AVERAGE = DISCHARGE_REF.groupby(DISCHARGE_REF.month).mean()
    # Runoff
    RUNOFF_AVERAGE = RUNOFF_REF.groupby(RUNOFF_REF.month).mean()
    # Precip
    PRECIP_AVERAGE = PRECIP_REF.groupby(PRECIP_REF.month).mean()
    # HydroSOS Category
    hydroSOS = defineHydroSOScategory(RUNOFF_SELECTED,RUNOFF_AVERAGE,'runoff')
    # Calculate the anomaly
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'discharge'] = (DISCHARGE_SELECTED.query('month == @month_analyis & year == @year_analysis')['discharge'].item() - DISCHARGE_AVERAGE.query('month == @month_analyis')['discharge'].item()) / DISCHARGE_AVERAGE.query('month == @month_analyis')['discharge'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'runoff'] = (RUNOFF_SELECTED.query('month == @month_analyis & year == @year_analysis')['runoff'].item()- RUNOFF_AVERAGE.query('month == @month_analyis')['runoff'].item()) / RUNOFF_AVERAGE.query('month == @month_analyis')['runoff'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'precip'] = (PRECIP_SELECTED.query('month == @month_analyis & year == @year_analysis')['precip'].item() - PRECIP_AVERAGE.query('month == @month_analyis')['precip'].item()) / PRECIP_AVERAGE.query('month == @month_analyis')['precip'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'precip_val'] = PRECIP_SELECTED.query('month == @month_analyis & year == @year_analysis')['precip'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'hydroSOS_category'] = hydroSOS.query('month == @month_analyis & year == @year_analysis')['percentile_range'].item()

In [11]:
HTML(AVERAGE_PERCENTAGE.head(6).to_html(index=False))

codigo,discharge,runoff,precip,precip_val,hydroSOS_category
10,-0.932153,-0.911884,-0.572152,39.263125,Low flow
11,-0.995267,-0.997354,-0.706225,25.975625,Low flow
12,-0.997653,-0.998094,-0.703833,28.641000,Low flow
13,-0.990885,-0.969790,-0.601227,38.953600,Low flow
14,-0.975218,-0.972767,-0.729096,21.726500,Below normal
15,-0.979956,-0.976305,-0.720468,24.271667,Below normal


In [21]:
HTML(AVERAGE_PERCENTAGE.tail(6).to_html(index=False))

codigo,discharge,runoff,precip,precip_val,hydroSOS_category
63,-0.999999,-0.999999,-0.928909,6.249650,Low flow
64,-1.000000,-1.000000,-0.924420,6.564550,Low flow
65,-0.998766,-0.998849,-0.965479,3.052156,Low flow
66,-1.000000,-1.000000,-0.923693,6.604375,Low flow
67,-0.999997,-0.999998,-0.930290,6.115100,Low flow
68,-0.999998,-0.999999,-0.932943,5.918000,Low flow


In [12]:
## Agregado por rrodriguez

In [23]:
# Cargar shapefile
cuencas_n2_path = 'C:/WORK_RR_SSP/GIS/GENERAL/BASE_OAN/Datos/c098/c098Polygon.shp'
cuencas_n2 = gpd.read_file(cuencas_n2_path)

In [24]:
codigos_cuencas = []
for cod in cuencas_n2["codcuenca"]:
        codigo = str(cod).split('.')[0]
        codigos_cuencas.append(codigo)

In [28]:
cuencas_n2['cod_cuenca'] = codigos_cuencas
cuencas_n2['discharge'] = AVERAGE_PERCENTAGE['discharge']*100
cuencas_n2['runoff'] = AVERAGE_PERCENTAGE['runoff']
cuencas_n2['precip'] = AVERAGE_PERCENTAGE['precip_val']
cuencas_n2['hydroSOS_c'] = AVERAGE_PERCENTAGE['hydroSOS_category']

In [29]:
cuencas_n2

,codcuenca,area,nombrec2,popup,cabecera,geometry,cod_cuenca,discharge,runoff,precip,hydroSOS_c
0,10.0,8222,RÍO CUAREIM,RÍO CUAREIM,10.0,"POLYGON ((-56.87237 -30.08809, -56.87239 -30.0...",10,-93.215335,-0.911884,39.263125,Low flow
1,11.0,2583,RÍO URUGUAY entre Río Cuareim y Río Arapey Grande,RÍO URUGUAY entre Río Cuareim y Río Arapey Grande,10.0,"POLYGON ((-57.62426 -30.18712, -57.62397 -30.1...",11,-99.526724,-0.997354,25.975625,Low flow
2,12.0,2152,RÍO ARAPEY CHICO,RÍO ARAPEY CHICO,12.0,"POLYGON ((-57.14899 -30.72436, -57.14366 -30.7...",12,-99.765316,-0.998094,28.641000,Low flow
3,13.0,9698,RÍO ARAPEY GRANDE,RÍO ARAPEY GRANDE,12.0,"POLYGON ((-57.44268 -30.88086, -57.44480 -30.8...",13,-99.088549,-0.969790,38.953600,Low flow
4,14.0,1631,RÍO URUGUAY entre Río Arapey Grande y Río Daymán,RÍO URUGUAY entre Río Arapey Grande y Río Daymán,10.0,"POLYGON ((-57.86116 -30.93648, -57.86087 -30.9...",14,-97.521770,-0.972767,21.726500,Below normal
5,15.0,3415,RÍO DAYMÁN,RÍO DAYMÁN,15.0,"POLYGON ((-57.86418 -31.36205, -57.86099 -31.3...",15,-97.995552,-0.976305,24.271667,Below normal
6,16.0,1714,RÍO URUGUAY entre Río Daymán y Río Queguay Grande,RÍO URUGUAY entre Río Daymán y Río Queguay Grande,14.0,"POLYGON ((-58.01106 -31.50361, -58.00827 -31.5...",16,-97.113336,-0.966200,14.273200,Below normal
7,17.0,8549,RÍO QUEGUAY GRANDE,RÍO QUEGUAY GRANDE,17.0,"POLYGON ((-56.27776 -31.74131, -56.27528 -31.7...",17,-81.492710,-0.898772,41.322600,Below normal
8,18.0,3729,RÍO URUGUAY entre Río Queguay Grande y Río Negro,RÍO URUGUAY entre Río Queguay Grande y Río Negro,14.0,"POLYGON ((-58.08290 -32.14423, -58.07632 -32.1...",18,-99.575222,-0.998171,13.859600,Low flow
9,19.0,3636,RÍO URUGUAY entre Río Negro y Río de la Plata,RÍO URUGUAY entre Río Negro y Río de la Plata,19.0,"POLYGON ((-57.96422 -33.40336, -57.96257 -33.4...",19,-99.819516,-0.999308,3.463575,Low flow


In [30]:
# Specify the output file path and name
output_path = 'C:/WORK_RR_SSP/Sala_Situacion_Dinagua/balance_hidrico regional_30_06_2023/output_modelo/shape_anomalia_esc/'
output_name = 'Esc_anom_junio_2023_cuencas_n2.shp'


# Save the DataFrame as a new shapefile
cuencas_n2.to_file(output_path+output_name)

In [12]:
# AVERAGE_PERCENTAGE.to_clipboard(index=False)

## Calculate the percentage of anomaly for each basin code level 2 and a specific year

In [13]:
#--
year_analysis = 2019

AVERAGE_PERCENTAGE_YEAR = pd.DataFrame()
AVERAGE_PERCENTAGE_YEAR['codigo'] = BASIN_LEVEL2.columns
AVERAGE_PERCENTAGE_YEAR['runoff'] = np.nan
AVERAGE_PERCENTAGE_YEAR['runoff_anomaly'] = np.nan
AVERAGE_PERCENTAGE_YEAR['runoff_mean'] = np.nan



for basin in BASIN_LEVEL2.columns:
    # filter the basin code bas
    filter_col = [col for col in DISCHARGE if col.startswith(str(basin))]
    # Select basins level 3 that belongs to level 2 basin
    # Runoff
    RUNOFF_FILTER = RUNOFF[filter_col]
    RUNOFF_SELECTED = pd.DataFrame()
    RUNOFF_SELECTED['year'] = RUNOFF['year']
    RUNOFF_SELECTED['month'] = RUNOFF['month']    
    RUNOFF_SELECTED['runoff'] = RUNOFF_FILTER.mean(axis=1)
    RUNOFF_REF = RUNOFF_SELECTED[(RUNOFF_SELECTED['year'] >= year_start) & (RUNOFF_SELECTED['year'] <= year_end)]

    sum_avg_runoff_year = RUNOFF_SELECTED.groupby(['year'])['runoff'].sum()
    sum_avg_runoff_year = sum_avg_runoff_year.to_frame()

    # Runoff
    RUNOFF_AVERAGE = RUNOFF_REF.groupby(['year'])['runoff'].sum()
    RUNOFF_AVERAGE = RUNOFF_AVERAGE.to_frame()

    # Calculate the anomaly
    AVERAGE_PERCENTAGE_YEAR.loc[AVERAGE_PERCENTAGE_YEAR.eval('codigo==@basin'),'runoff'] = sum_avg_runoff_year.query('year == @year_analysis')['runoff'].item()
    AVERAGE_PERCENTAGE_YEAR.loc[AVERAGE_PERCENTAGE_YEAR.eval('codigo==@basin'),'runoff_anomaly'] = (sum_avg_runoff_year.query('year == @year_analysis')['runoff'].item() - RUNOFF_AVERAGE['runoff'].mean()) / RUNOFF_AVERAGE['runoff'].mean()
    AVERAGE_PERCENTAGE_YEAR.loc[AVERAGE_PERCENTAGE_YEAR.eval('codigo==@basin'),'runoff_mean'] = RUNOFF_AVERAGE['runoff'].mean()

In [14]:
HTML(AVERAGE_PERCENTAGE_YEAR.head(6).to_html(index=False))

codigo,runoff,runoff_anomaly,runoff_mean
10,887.031000,0.588473,558.417338
11,798.323875,0.709706,466.936418
12,770.736250,0.513653,509.189507
13,771.396390,0.579753,488.301809
14,647.518550,0.764475,366.975182
15,675.509533,0.605171,420.833452
